In [ ]:
import json
import numpy as np
import os
import psycopg2
import psycopg2.extensions

from py2neo import Graph
from twitter import Twitter, OAuth

with open('config.json') as config_file:
    config = json.load(config_file)

graph = Graph(config["neo4j"])

if 'http_proxy' in os.environ:
    if 'https_proxy' not in os.environ:
        os.environ["https_proxy"] = os.environ["http_proxy"]

twitter = Twitter(auth=OAuth(config['token'], config['token_secret'], config['consumer_key'], config['consumer_secret']))

field_names =  ['user_screen_name', 'user_name', 'id_str', 'created_at', 
                'sentiment', 'categories', 'text', 'klout_score', 'segment_id',
               'got_reply', 'followers_count', 'is_agent_reply']

segment_dict = {0:'-', 1:'Mass Market', 2:'Young Professional',
                3:'Mass Afluent', 4:'Affluent', 5:'High Net Worth'}

def get_priority_key(tw_l):
    for tw in tw_l:
        if not tw['is_agent_reply']:
            p = (tw['segment_id']/2 - tw['sentiment'] - tw['got_reply']*5 +
                tw['followers_count']/1000 + int('fraud' in tw['categories']))
            return p

In [ ]:
from flask import Flask, render_template, jsonify, request, send_file
from flask_sockets import Sockets
from json import dumps
import gevent
import time
from gevent.pool import Pool
import StringIO

app = Flask(__name__)
sockets = Sockets(app)

@sockets.route('/twitter_stream')
def twitter_stream(ws):
    j = 0
    
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    curs.execute("LISTEN new_tweet;")
    
    while not ws.closed:
        conn.poll()
        t0 = time.time()
        if conn.notifies:
            notify = conn.notifies.pop(0)
            
            ws.send("{notification:\'new tweet\'}")
            j+=1
        if time.time() - t0 > 3600:
            print("1hr timeout")
            break
        else:
            gevent.sleep()
    conn.close()

@app.route('/reply', methods=['POST'])
def profile():
    data = eval(request.form['py_data'])
    status = request.form['status']
    status_id = request.form['status_id']
    
    sentiment = data['sentiment']
    user_name = data['user_name']
    
    tweet = twitter.statuses.update(status=status, in_reply_to_status_id=status_id)
    
    conn = psycopg2.connect(database=config["twitter_db"]["database"], user=config["twitter_db"]["user"])
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    
    curs.execute("""update customers
                set got_reply=1
                where user_screen_name=%s""",(tweet['in_reply_to_screen_name'],))
    
    curs.execute("""insert into stream (id_str, text, created_at, in_reply_to_status_id,
                in_reply_to_screen_name, user_screen_name, user_name, sentiment, is_agent_reply)
                values (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                 (tweet['id_str'],
                  tweet['text'],
                  tweet['created_at'],
                  tweet['in_reply_to_status_id'],
                  tweet['in_reply_to_screen_name'],
                  tweet['in_reply_to_screen_name'],
                  user_name,
                  sentiment,
                  1,))
    
    conn.close()

    return 'good'
    
@app.route('/data', methods=['GET'])
def provide_data():
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()

    curs.execute("""select stream.user_screen_name, stream.user_name, stream.id_str,
                stream.created_at, stream.sentiment, stream.categories, stream.text,
                customers.klout_score, customers.segment, customers.got_reply, 
                customers.followers_count, stream.is_agent_reply
                from stream
                INNER JOIN customers
                ON customers.user_screen_name=stream.user_screen_name
                where stream.user_screen_name in 
                (
                    select tmp.user_screen_name
                    from (
                        select user_screen_name as user_screen_name, max(id_str) as max_id_str
                        from stream
                        group by user_screen_name
                    ) tmp
                    left join (
                        select user_screen_name, got_reply
                        from customers
                    ) tmp2
                    on tmp.user_screen_name=tmp2.user_screen_name
                    order by tmp2.got_reply asc, tmp.max_id_str desc limit 10
                )
                order by customers.got_reply desc, stream.id_str desc
                ;""")
    rec = curs.fetchall()
    conn.close()
    
    user_blocks = {v[0]:[] for v in rec}
    for row in rec:
        if len(user_blocks[row[0]]) < 10:
            user_blocks[row[0]] += [{k:v for k,v in zip(field_names, row)}]
            user_blocks[row[0]][-1]['id_str'] = str(user_blocks[row[0]][-1]['id_str'])
            user_blocks[row[0]][-1]['segment'] = segment_dict[user_blocks[row[0]][-1]['segment_id']]
    
    user_blocks_v = user_blocks.values()
    user_blocks_v_s = sorted(user_blocks_v, key=get_priority_key)
    
    return jsonify({'data':user_blocks_v_s})

@app.route('/graph')
def get_graph():
    results = graph.cypher.execute(
        "match (p)-[r]->(p1) "
        "where ((r.favorited>0 and r.retweeted>0) or (r.favorited>0 and r.replied>0) or (r.retweeted>0 and r.replied>0) "
        "or (r.favorited>1) or (r.retweeted>1) or (r.replied>1)) and (not p=p1) "
        "return p.screen_name, sum(r.favorited)+sum(r.retweeted)+sum(r.replied) as cardinality, p1.screen_name "
        "limit 500;")
    nodes = []
    rels = []

    persons = {}
    for person_b, cardinality, person_a in results:
        if person_a in persons:
            persons[person_a] += 1
        else:
            persons[person_a] = 1

        if person_b not in persons:
            persons[person_b] = 1

    for i, v in persons.iteritems():
        if i == u'gunjan_amit':
            person = {
                "id": persons.keys().index(i),
                "caption": i, 
                "role": "center"}
        else:
            person = {
                "id": persons.keys().index(i),
                "caption": i, 
                "role": "customer" if np.random.rand()>0.9 else "user",
                }
        nodes.append(person)

    for person_b, cardinality, person_a in results:
        person = {"caption": person_a, "role": "customer"}
        source = persons.keys().index(person_a)
        target = persons.keys().index(person_b)
        rels.append({"source": source, 
            "target": target, 
            "weight": cardinality,
            "caption": "Value: {0}".format(cardinality)
            })

    return jsonify({"comment": "Twitter users graph", "nodes": nodes, "edges": rels})

@app.route('/download/customers')
def index():
    strIO = StringIO.StringIO()
    strIO.write('Customer ID, NBO\n')
    for i in range(10):
        strIO.write(str(np.floor(10000*np.random.rand()))+',Credit Card\n')
    
    strIO.seek(0)
    return send_file(strIO,
                     attachment_filename="Customers.csv",
                     as_attachment=True)

@app.route('/map/get_customers/<state>')
def map_get_customers(state):
    # Postgre request
    
    
    response = {"data": [
        [
          "0",
          "Tiger Nixon",
          "System Architect",
          "Edinburgh"
        ],
        [
          "1",
          "Tiger Nixon 1",
          "System Architect",
          "Edinburgh"
        ],
        [
          "2",
          "Tiger Nixon 2",
          "System Architect",
          "Edinburgh"
        ],
        [
          "1",
          "Tiger Nixon",
          "System Architect",
          "Edinburgh"
        ],
        [
          "1",
          "Tiger Nixon",
          "System Architect",
          "Edinburgh"
        ],
        [
          "1",
          "Tiger Nixon",
          "System Architect",
          "Edinburgh"
        ],
        [
          "1",
          "Tiger Nixon",
          "System Architect",
          "Edinburgh"
        ],
        [
          "1",
          "Tiger Nixon",
          "System Architect",
          "Edinburgh"
        ],
        [
          "1",
          "Tiger Nixon",
          "System Architect",
          "Edinburgh"
        ],
        [
          "1",
          "Tiger Nixon",
          "System Architect",
          "Edinburgh"
        ]]}
    return jsonify(response)

@app.route('/map')
def mappage():
    try:
        return render_template("map.html")
    except Exception, e:
        return str(e)

@app.route('/cluster')
def clusterpage():
    try:
        return render_template("cluster.html")
    except Exception, e:
        return str(e)

@app.route('/')
def homepage():
    try:
        return render_template("home.html")
    except Exception, e:
        return str(e)



if __name__ == "__main__":
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    server = pywsgi.WSGIServer(('', 8090), app, handler_class=WebSocketHandler, spawn=Pool())
    server.serve_forever()

ERROR:httpstream:! SocketError: Unknown error
ERROR:__main__:Exception on /graph [GET]
Traceback (most recent call last):
  File "D:\bin\Anaconda2\lib\site-packages\flask\app.py", line 1817, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\bin\Anaconda2\lib\site-packages\flask\app.py", line 1477, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\bin\Anaconda2\lib\site-packages\flask\app.py", line 1381, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\bin\Anaconda2\lib\site-packages\flask\app.py", line 1475, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\bin\Anaconda2\lib\site-packages\flask\app.py", line 1461, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-52a97de55ec9>", line 119, in get_graph
    results = graph.cypher.execute(
  File "D:\bin\Anaconda2\lib\site-packages\py2neo\core.py", line 661, in cypher
    metadata = self.resource.m